<a href="https://colab.research.google.com/github/alexngocvu/dbclass/blob/main/Database_group_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Create sample tables
c.execute('''
CREATE TABLE Departments (
    DepartmentID INTEGER PRIMARY KEY,
    DepartmentName TEXT
);
''')

c.execute('''
CREATE TABLE Projects (
    ProjectID INTEGER PRIMARY KEY,
    ProjectName TEXT,
    DepartmentID INTEGER,
    FOREIGN KEY (DepartmentID) REFERENCES Departments(DepartmentID)
);
''')

c.execute('''
CREATE TABLE Employees (
    EmployeeID INTEGER PRIMARY KEY,
    Name TEXT,
    Position TEXT,
    DepartmentID INTEGER,
    Salary REAL,
    FOREIGN KEY (DepartmentID) REFERENCES Departments(DepartmentID)
);
''')

c.execute('''
CREATE TABLE Assignments (
    EmployeeID INTEGER,
    ProjectID INTEGER,
    HoursWorked INTEGER,
    PRIMARY KEY (EmployeeID, ProjectID),
    FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID),
    FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID)
);
''')

# Insert 10 rows of sample data into each table
departments_data = [(1, 'HR'), (2, 'Engineering')]
projects_data = [(1, 'Project A', 1), (2, 'Project B', 2)]
employees_data = [
    (1, 'Alice', 'HR', 1, 70000),
    (2, 'Bob', 'Engineer', 2, 80000),
    (3, 'Charlie', 'HR', 1, 75000),
    (4, 'David', 'Engineer', 2, 85000),
    (5, 'Eva', 'HR', 1, 72000),
    (6, 'Frank', 'Engineer', 2, 90000),
    (7, 'Grace', 'HR', 1, 78000),
    (8, 'Henry', 'Engineer', 2, 82000),
    (9, 'Ivy', 'HR', 1, 76000),
    (10, 'Jack', 'Engineer', 2, 88000)
]
assignments_data = [
    (1, 1, 20), (2, 2, 15), (3, 1, 25), (4, 2, 30),
    (5, 1, 18), (6, 2, 22), (7, 1, 28), (8, 2, 32),
    (9, 1, 24), (10, 2, 26)
]

c.executemany('INSERT INTO Departments VALUES (?, ?)', departments_data)
c.executemany('INSERT INTO Projects VALUES (?, ?, ?)', projects_data)
c.executemany('INSERT INTO Employees VALUES (?, ?, ?, ?, ?)', employees_data)
c.executemany('INSERT INTO Assignments VALUES (?, ?, ?)', assignments_data)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [12]:
#Question 4.1 (part 1) - Vu Nguyen Bao Ngoc (Student ID: 12225193)
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Execute the SQL query
c.execute('''
    SELECT Employees.Name AS EmployeeName, Projects.ProjectName AS ProjectName
    FROM Employees
    JOIN Assignments ON Employees.EmployeeID = Assignments.EmployeeID
    JOIN Projects ON Assignments.ProjectID = Projects.ProjectID
''')

# Fetch the results into a DataFrame
rows = c.fetchall()
df = pd.DataFrame(rows, columns=['EmployeeName', 'ProjectName'])

# Print the DataFrame as a table
print(df)

# Commit the changes and close the connection
conn.commit()
conn.close()



  EmployeeName ProjectName
0        Alice   Project A
1          Bob   Project B
2      Charlie   Project A
3        David   Project B
4          Eva   Project A
5        Frank   Project B
6        Grace   Project A
7        Henry   Project B
8          Ivy   Project A
9         Jack   Project B


In [13]:
#Question 4.1 (part 2) - Vu Nguyen Bao Ngoc (Student ID: 12225193)
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect('database.db')
c = conn.cursor()

# Execute the SQL query to find the department with the highest average salary
c.execute('''
    SELECT DepartmentName, AVG(Salary) AS AvgSalary
    FROM Employees
    JOIN Departments ON Employees.DepartmentID = Departments.DepartmentID
    GROUP BY Departments.DepartmentID
    HAVING AVG(Salary) = (
        SELECT MAX(AvgSal)
        FROM (
            SELECT AVG(Salary) AS AvgSal
            FROM Employees
            GROUP BY DepartmentID
        )
    );
''')

# Fetch the results into a DataFrame
rows = c.fetchall()
df = pd.DataFrame(rows, columns=['DepartmentName', 'AvgSalary'])

# Print the DataFrame as a table
print(df)

# Commit the changes and close the connection
conn.commit()
conn.close()

  DepartmentName  AvgSalary
0    Engineering    85000.0


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('database.db')
c = conn.cursor()

# Execute the SELECT statement with the RANK() window function
c.execute('''
SELECT
    EmployeeID,
    Name,
    Position,
    DepartmentID,
    Salary,
    RANK() OVER (ORDER BY Salary DESC) AS SalaryRank
FROM Employees;
''')

# Fetch all the results
result = c.fetchall()

# Create a DataFrame from the result
columns = ['EmployeeID', 'Name', 'Position', 'DepartmentID', 'Salary', 'SalaryRank']
df = pd.DataFrame(result, columns=columns)

# Print the DataFrame
print(df)

# Commit the changes and close the connection
conn.commit()
conn.close()


   EmployeeID     Name  Position  DepartmentID   Salary  SalaryRank
0           6    Frank  Engineer             2  90000.0           1
1          10     Jack  Engineer             2  88000.0           2
2           4    David  Engineer             2  85000.0           3
3           8    Henry  Engineer             2  82000.0           4
4           2      Bob  Engineer             2  80000.0           5
5           7    Grace        HR             1  78000.0           6
6           9      Ivy        HR             1  76000.0           7
7           3  Charlie        HR             1  75000.0           8
8           5      Eva        HR             1  72000.0           9
9           1    Alice        HR             1  70000.0          10
